### Importing relevant Libraries for  the project

In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


### Reading the Csv file "flights.csv" and  printing head

In [98]:
df = pd.read_csv('data/flight.csv')
df.head()

,Unnamed: 0,acc.date,type,reg,operator,fat,location,dmg
0,0,3 Jan 2022,British Aerospace 4121 Jetstream 41,ZS-NRJ,SA Airlink,0,near Venetia Mine Airport,sub
1,1,4 Jan 2022,British Aerospace 3101 Jetstream 31,HR-AYY,LANHSA - Línea Aérea Nacional de Honduras S.A,0,Roatán-Juan Manuel Gálvez International Airpor...,sub
2,2,5 Jan 2022,Boeing 737-4H6,EP-CAP,Caspian Airlines,0,Isfahan-Shahid Beheshti Airport (IFN),sub
3,3,8 Jan 2022,Tupolev Tu-204-100C,RA-64032,"Cainiao, opb Aviastar-TU",0,Hangzhou Xiaoshan International Airport (HGH),w/o
4,4,12 Jan 2022,Beechcraft 200 Super King Air,NaN,private,0,"Machakilha, Toledo District, Grahem Creek area",w/o


df info for more details about  the data


In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2500 non-null   int64 
 1   acc.date    2500 non-null   object
 2   type        2500 non-null   object
 3   reg         2408 non-null   object
 4   operator    2486 non-null   object
 5   fat         2488 non-null   object
 6   location    2500 non-null   object
 7   dmg         2500 non-null   object
dtypes: int64(1), object(7)
memory usage: 156.4+ KB


In [100]:
#checking out df columns 
df.columns

Index(['Unnamed: 0', 'acc.date', 'type', 'reg', 'operator', 'fat', 'location',
       'dmg'],
      dtype='object')

## Making Copy of `df` for the Cleaning and renamed it to `dfc`

Cleaning Columns  

In [101]:
# Making  the copy of  the data form cleaning
dfc = df.copy()

In [102]:
# cleaning columns i.e striping spaces and replacing them with underscore and and getting to lowercase
dfc.columns = dfc.columns.str.strip().str.replace(" ", "_").str.lower()


In [103]:
dfc.columns


Index(['unnamed:_0', 'acc.date', 'type', 'reg', 'operator', 'fat', 'location',
       'dmg'],
      dtype='object')

### Dropping `unnamed:_01` column: 
 The column seems to be a duplicated indexing maybe as a result of concat. I will drop it as we have a proper indexing showing all the 2500 entries 

In [104]:
#Unnamed column seems to be  duplicated indexing and every index exists twice
dfc['unnamed:_0'].value_counts()

unnamed:_0
1249    2
0       2
1233    2
1232    2
1231    2
       ..
6       2
5       2
4       2
3       2
2       2
Name: count, Length: 1250, dtype: int64

In [105]:
# Dropping dfc['unnamed:_0']
dfc = dfc.drop(columns=["unnamed:_0"])


In [106]:
dfc.columns

Index(['acc.date', 'type', 'reg', 'operator', 'fat', 'location', 'dmg'], dtype='object')

In [107]:
#Renaming 'acc.date' according to rules(snakecase)
dfc = dfc.rename(columns={'acc.date': 'acc_date'})


In [108]:
dfc.columns

Index(['acc_date', 'type', 'reg', 'operator', 'fat', 'location', 'dmg'], dtype='object')

## Checking Missing or Duplicated observations and values.

In [109]:
# Checking for any  missing values and observations 
dfc.isna().any()

acc_date    False
type        False
reg          True
operator     True
fat          True
location    False
dmg         False
dtype: bool

In [110]:
# cumulative total for  any missing values and observations
dfc.isna().sum()

acc_date     0
type         0
reg         92
operator    14
fat         12
location     0
dmg          0
dtype: int64

In [111]:
#Checking for each column type
dfc.dtypes

acc_date    object
type        object
reg         object
operator    object
fat         object
location    object
dmg         object
dtype: object

## Cleaning `acc_date` Column  
Changing `acc_date` to date format

In [112]:
# Convert to datetime; NaT for missing
dfc["acc_date"] = pd.to_datetime(dfc["acc_date"], errors="coerce")



#### Replacing missing dates
    We replace with Unkown for clean data then  later drop them for  times based analysis

In [113]:
dfc["acc_date"] = dfc["acc_date"].fillna("Unknown")


In [114]:
dfc.head(2)

,acc_date,type,reg,operator,fat,location,dmg
0,2022-01-03 00:00:00,British Aerospace 4121 Jetstream 41,ZS-NRJ,SA Airlink,0,near Venetia Mine Airport,sub
1,2022-01-04 00:00:00,British Aerospace 3101 Jetstream 31,HR-AYY,LANHSA - Línea Aérea Nacional de Honduras S.A,0,Roatán-Juan Manuel Gálvez International Airpor...,sub


## Cleaning `fat` Column
 Converting Number of fatalities `fat` to numeric and Cleaning `NaN`

In [115]:
dfc["fat"] = pd.to_numeric(dfc["fat"], errors="coerce")


In [116]:
#Checking for each column type
dfc.dtypes

acc_date     object
type         object
reg          object
operator     object
fat         float64
location     object
dmg          object
dtype: object

In [117]:
# dfc description 
dfc['fat'].describe()

count    2450.000000
mean        1.860408
std        13.182859
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       257.000000
Name: fat, dtype: float64

In [118]:
#After converting the column to numeric we have a   cumulative  total of 50 missing vales meaning some of  the values were either spaces  or error values  that were dispatched 
dfc["fat"].count()


np.int64(2450)

In [119]:
dfc["fat"].isna().sum()


np.int64(50)

### Dropping the missing values in fat
I have  decided  to drop all  the missing values for  the following reasons:
- Missing fat means unknown, not zero
- Dropping preserves data integrity
- Imputing would invent fatalities
- Mean/median are misleading for count data i.e median is  0 and mean 1.8
- Fatalities are highly skewed with outliers
- Missing values are very few (~2%)
- Aggregations (sum, mean) remain trustworthy

In [120]:
#Dropping
dfc = dfc.dropna(subset=["fat"])


In [121]:
#Confirming
dfc["fat"].isna().sum()

np.int64(0)

In [122]:
dfc.columns

Index(['acc_date', 'type', 'reg', 'operator', 'fat', 'location', 'dmg'], dtype='object')

## 

## Cleaning `type` Column


In [123]:
dfc.isna().sum()

acc_date     0
type         0
reg         88
operator    14
fat          0
location     0
dmg          0
dtype: int64

In [124]:
dfc["type"].nunique()
dfc["type"].value_counts()

type
Cessna 208B Grand Caravan                        110
Antonov An-2R                                     58
Beechcraft 200 Super King Air                     58
de Havilland Canada DHC-6 Twin Otter 300          34
Cessna 208 Caravan I                              30
                                                ... 
Convair CV-340                                     2
Dornier 228-202                                    2
IRMA/Pilatus Britten-Norman BN-2A-26 Islander      2
Boeing 777-2B5ER                                   2
Boeing 737-7L9 (WL)                                2
Name: count, Length: 523, dtype: int64

### Creating new column `plane_type`
This would  help ein cleaning the types column and getting  exact frequency of  each  plane types

In [125]:
#creating plane_type column 
dfc['plane_type'] =(
    dfc["type"]
    .str.strip()                     # remove leading or trailing spaces
    .str.replace(r"\s+", " ", regex=True)  # collapse multiple spaces
    .str.upper()                     # unify case
)



In [126]:
dfc.columns

Index(['acc_date', 'type', 'reg', 'operator', 'fat', 'location', 'dmg',
       'plane_type'],
      dtype='object')

In [127]:
dfc["plane_type"].value_counts()


plane_type
CESSNA 208B GRAND CARAVAN                        110
ANTONOV AN-2R                                     58
BEECHCRAFT 200 SUPER KING AIR                     58
DE HAVILLAND CANADA DHC-6 TWIN OTTER 300          34
CESSNA 208 CARAVAN I                              30
                                                ... 
CONVAIR CV-340                                     2
DORNIER 228-202                                    2
IRMA/PILATUS BRITTEN-NORMAN BN-2A-26 ISLANDER      2
BOEING 777-2B5ER                                   2
BOEING 737-7L9 (WL)                                2
Name: count, Length: 523, dtype: int64

Then type Column is Correctly  tabulated  and  doesnt need cleaning 

## Cleaning `reg` Column

In [128]:
dfc["reg"].value_counts()

reg
unreg.    6
5Y-SAV    4
P2-PXE    4
N817NW    4
N233SW    4
         ..
PR-YRH    2
N743QS    2
N771DX    2
N866JA    2
G-IPAX    2
Name: count, Length: 1171, dtype: int64

In [129]:
dfc.isna().sum()

acc_date       0
type           0
reg           88
operator      14
fat            0
location       0
dmg            0
plane_type     0
dtype: int64

Choose to rename the NaN in `reg` to `UNKNOWN`for  the following reasons:
- Makes missing registrations visible in counts and charts
- Improves readability in reports and tables
- Treats unknown registrations as a valid category


In [130]:
# filling  then NaNs with Unknown
dfc["reg"] = dfc["reg"].fillna("UNKNOWN")


In [131]:
dfc["reg"].value_counts()

reg
UNKNOWN    88
unreg.      6
VH-...      4
P2-PXE      4
5Y-SAV      4
           ..
PR-YRH      2
N743QS      2
N771DX      2
N866JA      2
G-IPAX      2
Name: count, Length: 1172, dtype: int64

In [132]:
dfc.isna().sum()

acc_date       0
type           0
reg            0
operator      14
fat            0
location       0
dmg            0
plane_type     0
dtype: int64

## Cleaning `operator` Column

In [133]:
dfc["operator"].value_counts()

operator
private                                      200
Delta Air Lines                               44
American Airlines                             44
Southwest Airlines                            36
United Airlines                               36
                                            ... 
América Latina Tecnologia Agrícola - ALTA      2
Starjet Inc.                                   2
Air Serv Limited                               2
Roper Aviation LLC                             2
Eclipse Transport LLC                          2
Name: count, Length: 813, dtype: int64

Renaming  the `NaNs` to `Unknown`  as the Operator was not reported
- Operator name is categorical, not numerical
- Missing operator means not reported, not invalid
- "Unknown" preserves the record without guessing
- Prevents silent exclusion of valid accidents

In [134]:
dfc["operator"] = dfc["operator"].fillna("Unknown")



In [135]:
#Confirming 
dfc.isna().sum()

acc_date      0
type          0
reg           0
operator      0
fat           0
location      0
dmg           0
plane_type    0
dtype: int64

## Cleaning `location` Column

In [136]:
dfc["location"].value_counts()

location
Chicago-O'Hare International Airport, IL (ORD/KORD)    28
Nashville-John C. Tune Airport, TN (KJWN)              24
Denver International Airport, CO (DEN/KDEN)            16
Houston-William P. Hobby Airport, TX (HOU)             14
Vancouver International Airport, BC (YVR/CYVR)         14
                                                       ..
Sanikiluaq Airport, NU (YSK)                            2
Dubuque, IA                                             2
Cancún Airport (CUN/MMUN)                               2
14,8 km NW of Shabunda                                  2
50 km N of Turks and Caicos Islands                     2
Name: count, Length: 1035, dtype: int64

The Column looks good and doesnt need any cleaning 

## Cleaning `dmg` Column

In [137]:
dfc["dmg"].value_counts()

dmg
sub    1318
w/o     674
non     334
min      94
unk      30
Name: count, dtype: int64

In [138]:
#Encoding the dmg levels to the 3 categories outlined plus  unknown 

dfc["dmg"] = dfc["dmg"].replace({
    "min": "non",      # merge minor into non
    "unk": "Unknown"   # keep unknown separately
})

In [139]:
dfc["dmg"].value_counts()

dmg
sub        1318
w/o         674
non         428
Unknown      30
Name: count, dtype: int64

## Dropping 6 Missing accident date rows
This is in case  we will need time based analysis. We cant recreate the accident dates

In [140]:
# dfc_time will have a clean copy of data that has no Nat values in the date column for time based analysis 
dfc_time = dfc.dropna(subset=["acc_date"]).copy()

In [141]:
dfc.dtypes

acc_date       object
type           object
reg            object
operator       object
fat           float64
location       object
dmg            object
plane_type     object
dtype: object

In [142]:
dfc_time.dtypes

acc_date       object
type           object
reg            object
operator       object
fat           float64
location       object
dmg            object
plane_type     object
dtype: object

## Exporting the data
- dfc for clean dataframe 
- dfc_time for times based analysis


In [143]:
# clean dfc
dfc.to_csv("data/cleaned_crashed_flights.csv", index=False)


In [144]:
# cleaned times based analysis dataframe
dfc_time.to_csv("data/cleaned_crashed_flights_with_time.csv", index=False)

